# Feature engineering

Colunas FUNIL_ASSINATURA_PIPEDRIVE_status até ATENDIMENTOS_AGENDA_Datas Atendimento Médico

In [5]:
import pandas as pd

data = pd.read_csv("../data/data-preprocessed.csv")

## FUNIL_ASSINATURA_PIPEDRIVE_status

Transforma dados da coluna em one not encoding

In [6]:
data['FUNIL_ASSINATURA_PIPEDRIVE_status'].value_counts()

won     544
lost    428
Name: FUNIL_ASSINATURA_PIPEDRIVE_status, dtype: int64

In [7]:
#one_hot_encoded = pd.get_dummies(data['FUNIL_ASSINATURA_PIPEDRIVE_status'], prefix='status')
#data = pd.concat([data, one_hot_encoded], axis=1)
#print(data)

     PESSOA_PIPEDRIVE_id_person  PESSOA_PIPEDRIVE_id_gender  \
0                           483                          64   
1                           860                          63   
2                           961                          63   
3                           972                          63   
4                           973                          64   
..                          ...                         ...   
967                        5944                          64   
968                        5948                          64   
969                        5949                          64   
970                        5954                          63   
971                        5956                          63   

     PESSOA_PIPEDRIVE_id_marrital_status PESSOA_PIPEDRIVE_state  \
0                                     80           Minas Gerais   
1                                     80                 Paraná   
2                                     80  

## FUNIL_ASSINATURA_PIPEDRIVE_start_of_service

Substitui valores nulos pela data de início do contrato e transforma o tipo em datetime

In [8]:
data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service']

0      2023-01-27
1      2022-06-21
2      2021-07-05
3      2021-06-28
4      2021-06-28
          ...    
967    2023-11-06
968    2023-11-07
969    2023-11-07
970    2023-11-07
971    2023-11-08
Name: FUNIL_ASSINATURA_PIPEDRIVE_start_of_service, Length: 972, dtype: object

In [9]:
data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'] = pd.to_datetime(data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'])


for indice, valor in data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'].items():
    if pd.isnull(valor):
        data.loc[indice, 'FUNIL_ASSINATURA_PIPEDRIVE_start_of_service'] = data.loc[indice, 'PESSOA_PIPEDRIVE_contract_start_date']
        
data['FUNIL_ASSINATURA_PIPEDRIVE_start_of_service']




0     2023-01-27
1     2022-06-21
2     2021-07-05
3     2021-06-28
4     2021-06-28
         ...    
967   2023-11-06
968   2023-11-07
969   2023-11-07
970   2023-11-07
971   2023-11-08
Name: FUNIL_ASSINATURA_PIPEDRIVE_start_of_service, Length: 972, dtype: datetime64[ns]

## FUNIL_ASSINATURA_PIPEDRIVE_lost_time

Utilizada para fazer uma nova coluna em conjunto com outras

In [10]:
data['FUNIL_ASSINATURA_PIPEDRIVE_lost_time']

0      2023-08-26
1      2023-09-20
2      2021-09-30
3      2023-10-16
4      2022-02-07
          ...    
967     Em aberto
968     Em aberto
969     Em aberto
970     Em aberto
971     Em aberto
Name: FUNIL_ASSINATURA_PIPEDRIVE_lost_time, Length: 972, dtype: object

## Coluna extra: Tempo de permanência

Coluna "stay_time" criada para calcular tempo total do usuário na plataforma. Utiliza lost_time - start_of_service ou lost_time - contract_start_date, se os dados de start_of_service forem nulos. 

## FUNIL_ASSINATURA_PIPEDRIVE_lost_reason

Faz one hot encoding para motivos de cancelamento de assinatura

In [11]:
data['FUNIL_ASSINATURA_PIPEDRIVE_lost_reason'].value_counts()

Outro                                                         601
[Assinatura] Desligamento                                     211
[Assinatura] Empresa cancelou o benefício da Ana               53
[Assinatura] Não quer seguir com a Ana                         49
[Assinatura] Precisou cortar custos                            24
[Assinatura] Cancelamento por inadimplência                    18
[Assinatura] Está sem tempo para conciliar os atendimentos     16
Name: FUNIL_ASSINATURA_PIPEDRIVE_lost_reason, dtype: int64

In [12]:
lost_reason_dummies = pd.get_dummies(data['FUNIL_ASSINATURA_PIPEDRIVE_lost_reason'], prefix='lost_reason')
data = pd.concat([data, lost_reason_dummies], axis=1)


## FUNIL_ONBOARDING_PIPEDRIVE_add_time

Tudo certo por aqui, nada a acrescentar

In [13]:
data['FUNIL_ONBOARDING_PIPEDRIVE_add_time']

0      2021-09-16 18:15:11
1      2022-06-21 23:48:22
2      2021-09-16 18:28:04
3      2021-09-16 18:04:29
4      2021-09-16 18:05:03
              ...         
967    2023-11-06 18:34:05
968    2023-11-07 12:22:13
969    2023-11-07 13:05:44
970    2023-11-07 20:34:02
971    2023-11-08 19:03:40
Name: FUNIL_ONBOARDING_PIPEDRIVE_add_time, Length: 972, dtype: object

## FUNIL_ONBOARDING_PIPEDRIVE_status

One-hot encoding aplicado nessa coluna

In [14]:
data['FUNIL_ONBOARDING_PIPEDRIVE_status'].value_counts()

won             579
lost            258
open             99
Não iniciado     36
Name: FUNIL_ONBOARDING_PIPEDRIVE_status, dtype: int64

In [15]:
data_status_encoded = pd.get_dummies(data['FUNIL_ONBOARDING_PIPEDRIVE_status'], prefix='Status')
data = pd.concat([data, data_status_encoded], axis=1)

## FUNIL_ONBOARDING_PIPEDRIVE_lost_reason

One hot encoding aplicado na coluna

In [16]:
data['FUNIL_ONBOARDING_PIPEDRIVE_lost_reason'].value_counts()

Outro                                                                747
[Onboarding] Não retornou aos contatos de resgate                    195
[Onboarding] Não tem interesse em seguir nas etapas do onboarding     18
[Associade] Cancelou assinatura                                       12
Name: FUNIL_ONBOARDING_PIPEDRIVE_lost_reason, dtype: int64

In [17]:
lost_reason_dummies = pd.get_dummies(data['FUNIL_ONBOARDING_PIPEDRIVE_lost_reason'], prefix='lost_reason')
data = pd.concat([data, lost_reason_dummies], axis=1)


## FUNIL_ONBOARDING_PIPEDRIVE_activities_count

Tudo certo por aqui, nada a acrescentar

In [18]:
data['FUNIL_ONBOARDING_PIPEDRIVE_activities_count']

0      0
1      0
2      0
3      2
4      0
      ..
967    0
968    0
969    0
970    0
971    0
Name: FUNIL_ONBOARDING_PIPEDRIVE_activities_count, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Qde Todos Atendimentos

Dados não fornecidos pela empresa, coluna será dropada

In [19]:
data['ATENDIMENTOS_AGENDA_Qde Todos Atendimentos']

0       2
1      25
2       0
3       0
4       0
       ..
967     0
968     1
969     0
970     2
971     0
Name: ATENDIMENTOS_AGENDA_Qde Todos Atendimentos, Length: 972, dtype: int64

In [20]:
data.drop('ATENDIMENTOS_AGENDA_Qde Todos Atendimentos', axis='columns', inplace=True)

## ATENDIMENTOS_AGENDA_Qde Atendimento Médico

Tudo certo por aqui, nada a acrescentar


In [21]:
data['ATENDIMENTOS_AGENDA_Qde Atendimento Médico']

0      0
1      1
2      0
3      0
4      0
      ..
967    0
968    0
969    0
970    0
971    0
Name: ATENDIMENTOS_AGENDA_Qde Atendimento Médico, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Faltas Atendimento Médico

Tudo certo por aqui, nada a acrescentar

In [22]:
data['ATENDIMENTOS_AGENDA_Faltas Atendimento Médico']


0      0
1      0
2      0
3      0
4      0
      ..
967    0
968    0
969    0
970    0
971    0
Name: ATENDIMENTOS_AGENDA_Faltas Atendimento Médico, Length: 972, dtype: int64

## ATENDIMENTOS_AGENDA_Datas Atendimento Médico

Tudo certo por aqui, nada a acrescentar

In [23]:
data['ATENDIMENTOS_AGENDA_Datas Atendimento Médico']

0            Nunca ocorreu
1      2023-01-12 17:00:00
2            Nunca ocorreu
3            Nunca ocorreu
4            Nunca ocorreu
              ...         
967          Nunca ocorreu
968          Nunca ocorreu
969          Nunca ocorreu
970          Nunca ocorreu
971          Nunca ocorreu
Name: ATENDIMENTOS_AGENDA_Datas Atendimento Médico, Length: 972, dtype: object